In [1]:
%load_ext autoreload
%autoreload 2

Clone the repo and install stuffs so it works on Google Collab. To run it Google Collab, uncomment everything in the cell below. To run it in your local machine, comment it

In [25]:
# !git clone https://github.com/longxvu/Adaptive_CA
# from google.colab import files
# import os
# import sys

# # To find local version of the library
# sys.path.append(os.path.join(".", 'Adaptive_CA'))
# %cd Adaptive_CA

# # Install necessary library
# !pip install openai

In [9]:
from assistant import GPTAssistant
from utils import show_json, list_all_assistant, get_api_key
from tool_functions import display_quiz_function_json
from openai import OpenAI
import pandas as pd
import json

## Init client and choosing our previously created assistant

In [11]:
# Create client and retrieve all current assistant
client = OpenAI(
    api_key=get_api_key()
)   # Maybe client should be outside assistant class but this will do for now

google.colab._shell


In [12]:
list_all_assistant(client)

id: asst_ZNz4lbi6z8bpKkE6APKdrpQ8
name: None
description: Assistant on helping children learn science concepts
instructions: You will be presented with a couple of dialogue between two cartoon characters.You need to identify a few science concepts presented in the dialogue to assist the children in learning the concepts.
model: gpt-3.5-turbo-0125
tools: [{'function': {'name': 'display_quiz', 'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.", 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string'}, 'questions': {'type': 'array', 'description': 'An array of questions, each with a title and potentially options (if multiple choice).', 'items': {'type': 'object', 'properties': {'question_text': {'type': 'string'}, 'question_type': {'type': 'string', 'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']}, 'difficulty': {'type': 'string', 'description': 'The difficulty level of each question. Easy difficu

In [ ]:
# update the function that assistant will use
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", tools=[{"type": "function", "function": display_quiz_function_json}])
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", instructions="You will be presented with a couple of dialogue between two cartoon characters."
#                                                                                           "You need to identify a few science concepts presented in the dialogue to assist the children in learning "
#                                                                                           "the concepts.")

In [13]:
assistant = GPTAssistant(client, assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8")

In [14]:
print(assistant.assistant.id)
print(assistant.thread.id)
print(assistant.get_all_messages())

asst_ZNz4lbi6z8bpKkE6APKdrpQ8
thread_J0pDBtBzaQViPhUUvkxN90K5
[]


## Get dialogue data from xlsx file

In [17]:
dialogue_file_path = "transcripts/town_picnic_base.xlsx"
df = pd.read_excel(dialogue_file_path)

In [18]:
transcripts = df[df["text"].notnull()]["text"].tolist()
base_question = df[df["text"].notnull()]["base_q"].tolist()

In [19]:
print(transcripts[0][0:500] + "...")
print("Base question:", base_question[0])

Elinor: The Town Picnic! [Elinor hops into view from the center. She giggles as she notices a bee and runs after it to a flower] Hmm? Oooh...
Ranger: [off-screen] C'mon, Elinor!
Elinor: Coming, Mom! [Elinor runs to the left, where Ranger follows behind her. Elinor runs to the entrance of the picnic grounds.] I just love the Town Picnic! What should we do first, Mom?
Ranger: I don't know, Elinor. Let's see... [gasp] Oh, we can do wood-carving with Mrs. Beaver. [Cut to Mrs. Beaver rapidly biting s...
Base question: Did you see that!?- the bee just drank some juice, called nectar from this flower! What do you think the bee is going to make with the juice from this flower?


## Main GPT dialogue loop

In [20]:
# Can set it to run forever but currently restricting to a limited amount of step for testing
max_step = 1
# num_questions = 3
step = 0
cur_dialogue_num = 0

In [21]:
# assistant.wait_on_run()
# tool_calls = assistant.last_run.required_action.submit_tool_outputs.tool_calls
# for tool_call in tool_calls:
#     print(json.loads(tool_call.function.arguments))

In [22]:
# assistant.resolve_run_required_action()
# assistant.get_all_messages()

First, the assistant is presented with the dialogue then it is asked to summarize it. Then we ask it to create a quiz of varying difficulties about the science concept presented in the dialogue

In [23]:
while step < max_step and cur_dialogue_num < len(transcripts):
    # Two steps: First, summarize the dialogue (for easier testing, can be omitted). Then ask the question about summarized dialogue
    assistant.submit_message(f"Summarize this dialogue: {transcripts[cur_dialogue_num]}.")
    print(assistant.get_last_response())
    assistant.submit_message(f"""
    Create a quiz with 3 questions to ask the child about the science concept presented in the dialogue. There should be one easy, one medium, and one hard question.
    Then, based on their answer, identify what did the child get wrong and gave them feedback.
    """)
    assistant.wait_on_run()
    if assistant.run_requires_action():
        assistant.resolve_run_required_action()
    print(assistant.get_last_response())

    step += 1
    cur_dialogue_num += 1
    print("==============================================================================")

assistant: The dialogue revolves around Elinor and her family attending the Town Picnic. They discuss various activities to do at the picnic, such as wood-carving, hayrides, and watching Mr. Cat at the dunking booth. Elinor eventually joins her friends, Olive and Ari, in line for special ketchup. While waiting in line, they talk about the food and the anticipation of trying it. Elinor gets distracted by a bee and follows it, leading her to discover something dripping from a flower.

Some science concepts presented in the dialogue include:
1. Bee pollination and flower interaction
2. Food anticipation and digestion
3. Waiting in line and patience in nature
Quiz: Science Concepts Quiz

Questions: Easy: What was Ari waiting in line for at the Town Picnic?
a) Ice cream
b) Ketchup
c) Soda
d) Cotton candy
Answer: b

Questions: Medium: What did Elinor get distracted by while waiting in line with Olive and Ari?
Answer: Bee

Questions: Hard: Explain the significance of bee pollination in the co

In [24]:
# for msg in assistant.get_all_messages():
#     print(msg)